# Text Processing for Vietnamese Football News

Notebook này xử lý dữ liệu bóng đá từ MongoDB, bao gồm:
- Kết nối MongoDB
- Làm sạch và chuẩn hóa văn bản tiếng Việt
- Tách từ và loại bỏ stopwords
- Trích xuất entities (đội bóng, cầu thủ, giải đấu)
- Phân tích tần suất từ và n-grams

## 1. Import Libraries

In [57]:
import os
import json
import re
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
import certifi
from collections import Counter
from tqdm import tqdm
from itertools import islice
from dotenv import load_dotenv
load_dotenv()

True

In [58]:
# Text processing libraries - PyVi
try:
    from pyvi import ViTokenizer, ViPosTagger
    PYVI_AVAILABLE = True
    print("✓ PyVi đã được cài đặt")
except ImportError:
    print("✗ PyVi chưa cài đặt. Cài đặt bằng: pip install pyvi")
    PYVI_AVAILABLE = False

✓ PyVi đã được cài đặt


In [59]:
# Text processing libraries - spaCy
try:
    import spacy
    nlp = spacy.load("vi_core_news_sm")
    SPACY_AVAILABLE = True
    print("✓ spaCy Vietnamese model đã được cài đặt")
except:
    print("✗ spaCy Vietnamese model chưa có. Cài đặt bằng: python -m spacy download vi_core_news_sm")
    SPACY_AVAILABLE = False

✗ spaCy Vietnamese model chưa có. Cài đặt bằng: python -m spacy download vi_core_news_sm


## 2. MongoDB Configuration

In [60]:
# MongoDB connection
MONGO_URI = load_dotenv("MONGO_URI") or os.getenv("MONGO_URI")
DB_NAME = "vnexpress_db"
COLLECTION_NAME = "vnexpress_bongda"

print(f"Database: {DB_NAME}")
print(f"Collection: {COLLECTION_NAME}")

Database: vnexpress_db
Collection: vnexpress_bongda


## 3. Vietnamese Text Processor Class

In [61]:
class VietnameseTextProcessor:
    def __init__(self):
        # Vietnamese stopwords
        self.stop_words = set([
            'và', 'của', 'trong', 'với', 'là', 'có', 'được', 'cho', 'từ', 'một', 'các',
            'để', 'không', 'sẽ', 'đã', 'về', 'hay', 'theo', 'như', 'cũng', 'này', 'đó',
            'khi', 'những', 'tại', 'sau', 'bị', 'giữa', 'trên', 'dưới', 'ngoài',
            'thì', 'nhưng', 'mà', 'hoặc', 'nếu', 'vì', 'do', 'nên', 'rồi', 'còn', 'đều',
            'chỉ', 'việc', 'người', 'lại', 'đây', 'đấy', 'ở', 'ra', 'vào', 'lên', 'xuống'
        ])
        
        # Football-specific entities
        self.football_teams = [
            'hà nội fc', 'hoàng anh gia lai', 'sài gòn fc', 'than quảng ninh', 'viettel fc',
            'becamex bình dương', 'slna', 'đà nẵng', 'nam định', 'hải phòng fc', 'hcm city',
            'song lam nghệ an', 'quảng nam', 'khánh hòa', 'đội tuyển việt nam', 'tuyển việt nam'
        ]
        
        self.player_patterns = [
            r'cầu thủ\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'tiền đạo\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'thủ môn\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'hậu vệ\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'tiền vệ\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'HLV\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'huấn luyện viên\s+([A-Za-zÀ-ỹ\s]{3,30})'
        ]
        
        self.competitions = [
            'v-league', 'v.league', 'v league', 'cup quốc gia', 'aff cup', 'sea games',
            'world cup', 'asian cup', 'champions league', 'afc cup', 'u23', 'u22', 'u19'
        ]
    
    def clean_text(self, text):
        """Làm sạch và chuẩn hóa text tiếng Việt"""
        if not text:
            return ""
        
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\sàáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđĐ]', ' ', text)
        text = text.lower()
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize_vietnamese(self, text):
        """Tách từ tiếng Việt"""
        if PYVI_AVAILABLE:
            try:
                return ViTokenizer.tokenize(text).split()
            except:
                pass
        return text.split()
    
    def remove_stopwords(self, tokens):
        return [token for token in tokens if token not in self.stop_words and len(token) > 1]
    
    def extract_entities(self, text):
        entities = {'teams': [], 'players': [], 'competitions': []}
        text_lower = text.lower()
        
        for team in self.football_teams:
            if team in text_lower:
                entities['teams'].append(team)
        
        for pattern in self.player_patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                player_name = match.group(1).strip()
                if len(player_name) > 3:
                    entities['players'].append(player_name)
        
        for comp in self.competitions:
            if comp in text_lower:
                entities['competitions'].append(comp)
        
        return entities
    
    def process_document(self, doc):
        title = doc.get('title', '')
        content = doc.get('content', '')
        full_text = f"{title} {content}"
        
        cleaned_text = self.clean_text(full_text)
        tokens = self.tokenize_vietnamese(cleaned_text)
        filtered_tokens = self.remove_stopwords(tokens)
        entities = self.extract_entities(full_text)
        
        return {
            'doc_id': str(doc.get('_id', '')),
            'title': title,
            'url': doc.get('url', ''),
            'date': doc.get('date', ''),
            'author': doc.get('author', ''),
            'original_length': len(full_text),
            'cleaned_text': cleaned_text,
            'tokens': tokens,
            'filtered_tokens': filtered_tokens,
            'token_count': len(tokens),
            'filtered_count': len(filtered_tokens),
            'entities': entities,
            'entity_count': sum(len(entities[key]) for key in entities)
        }

print("✓ VietnameseTextProcessor class đã được định nghĩa")

✓ VietnameseTextProcessor class đã được định nghĩa


## 4. Database Functions

In [62]:
def connect_to_database():
    """Kết nối tới MongoDB"""
    try:
        client = MongoClient(MONGO_URI, tls=True, tlsCAFile=certifi.where())
        db = client[DB_NAME]
        collection = db[COLLECTION_NAME]
        doc_count = collection.count_documents({})
        print(f"✓ Kết nối database thành công!")
        print(f"  Tổng số documents: {doc_count}")
        return client, db, collection
    except Exception as e:
        print(f"✗ Lỗi kết nối database: {e}")
        return None, None, None

print("✓ Database functions đã được định nghĩa")

✓ Database functions đã được định nghĩa


## 5. Document Processing Functions

In [63]:
def process_all_documents(collection, limit=None):
    """Xử lý tất cả documents từ collection"""
    processor = VietnameseTextProcessor()
    if limit:
        documents = collection.find().limit(limit)
        total_docs = limit
    else:
        documents = collection.find()
        total_docs = collection.count_documents({})
    
    print(f"Bắt đầu xử lý {total_docs} documents...")
    processed_results, error_count = [], 0
    
    for doc in tqdm(documents, desc="Processing documents", total=total_docs):
        try:
            result = processor.process_document(doc)
            processed_results.append(result)
        except Exception as e:
            error_count += 1
            print(f"Lỗi xử lý document {doc.get('_id', 'unknown')}: {e}")
    
    print(f"✓ Hoàn thành! Xử lý thành công: {len(processed_results)}, Lỗi: {error_count}")
    return processed_results

print("✓ process_all_documents function đã được định nghĩa")

✓ process_all_documents function đã được định nghĩa


## 6. Analysis Functions

In [64]:
def analyze_results(results):
    """Phân tích kết quả xử lý"""
    print("\nPHÂN TÍCH KẾT QUẢ:")
    print(f"Tổng số documents đã xử lý: {len(results)}")
    
    token_counts = [r['token_count'] for r in results]
    filtered_counts = [r['filtered_count'] for r in results]
    entity_counts = [r['entity_count'] for r in results]
    
    print(f"\nThống kê Tokens:")
    print(f"  - Trung bình tokens/document: {sum(token_counts)/len(token_counts):.1f}")
    print(f"  - Trung bình filtered tokens/document: {sum(filtered_counts)/len(filtered_counts):.1f}")
    print(f"  - Trung bình entities/document: {sum(entity_counts)/len(entity_counts):.1f}")
    
    all_teams, all_players, all_competitions = [], [], []
    for r in results:
        all_teams.extend(r['entities']['teams'])
        all_players.extend(r['entities']['players'])
        all_competitions.extend(r['entities']['competitions'])
    
    print(f"\nTop 10 Đội bóng được nhắc đến nhiều nhất:")
    for team, count in Counter(all_teams).most_common(10):
        print(f"  - {team}: {count} lần")
    
    print(f"\nTop 10 Cầu thủ được nhắc đến nhiều nhất:")
    for player, count in Counter(all_players).most_common(10):
        print(f"  - {player}: {count} lần")
    
    print(f"\nTop 10 Giải đấu được nhắc đến nhiều nhất:")
    for comp, count in Counter(all_competitions).most_common(10):
        print(f"  - {comp}: {count} lần")

print("✓ analyze_results function đã được định nghĩa")

✓ analyze_results function đã được định nghĩa


In [65]:
def save_results(results, filename="processed_data.json"):
    """Lưu kết quả ra file JSON"""
    # Tạo folder outputs nếu chưa có
    output_dir = os.path.join("d:\\data\\Search_Engine", "outputs")
    os.makedirs(output_dir, exist_ok=True)
    
    filepath = os.path.join(output_dir, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2, default=str)
    print(f"✓ Đã lưu kết quả vào {filepath}")

print("✓ save_results function đã được định nghĩa")

✓ save_results function đã được định nghĩa


## 7. Word Frequency Analysis

## 6.5. Stemming và POS Tagging

### Stemming - Chuẩn hóa từ về dạng gốc
### POS Tagging - Gắn nhãn từ loại (sử dụng HMM trong PyVi)

In [66]:
def apply_stemming_and_pos(text):
    """
    Áp dụng POS Tagging (sử dụng HMM model trong PyVi)
    và trích xuất từ gốc từ POS tags
    """
    if not PYVI_AVAILABLE:
        print("⚠️ PyVi chưa cài đặt, không thể thực hiện POS tagging")
        return None
    
    try:
        # POS Tagging với PyVi (sử dụng HMM model)
        pos_result = ViPosTagger.postagging(ViTokenizer.tokenize(text))
        
        words = pos_result[0]  # Danh sách từ đã tách
        pos_tags = pos_result[1]  # Danh sách nhãn từ loại
        
        # Tạo list các tuple (word, pos_tag)
        word_pos_pairs = list(zip(words, pos_tags))
        
        # Stemming đơn giản: loại bỏ các từ phụ và giữ lại từ chính
        stemmed_words = []
        important_pos = ['N', 'V', 'A', 'M']  # Danh từ, Động từ, Tính từ, Số
        
        for word, pos in word_pos_pairs:
            # Chỉ giữ lại từ có ý nghĩa quan trọng
            if any(pos.startswith(p) for p in important_pos):
                stemmed_words.append(word)
        
        return {
            'original_tokens': words,
            'pos_tags': pos_tags,
            'word_pos_pairs': word_pos_pairs,
            'stemmed_tokens': stemmed_words
        }
    
    except Exception as e:
        print(f"Lỗi khi xử lý POS tagging: {e}")
        return None

print("✓ apply_stemming_and_pos function đã được định nghĩa")

✓ apply_stemming_and_pos function đã được định nghĩa


In [67]:
def analyze_document_with_pos(collection, doc_id=None, limit=1):
    """
    Phân tích document với POS Tagging và Stemming
    """
    processor = VietnameseTextProcessor()
    
    if doc_id:
        from bson import ObjectId
        doc = collection.find_one({"_id": ObjectId(doc_id)})
        docs = [doc] if doc else []
    else:
        docs = list(collection.find().limit(limit))
    
    if not docs:
        print("Không tìm thấy document!")
        return None
    
    results = []
    
    for doc in docs:
        print("\n" + "="*80)
        print(f"📄 PHÂN TÍCH DOCUMENT")
        print("="*80)
        print(f"Title: {doc.get('title', 'N/A')}")
        print(f"URL: {doc.get('url', 'N/A')}")
        print(f"Date: {doc.get('date', 'N/A')}")
        
        # Xử lý cơ bản
        basic_result = processor.process_document(doc)
        
        # Lấy cleaned text
        cleaned_text = basic_result['cleaned_text'][:1000]  # Giới hạn 1000 ký tự để demo
        
        print(f"\n📝 Text mẫu (1000 ký tự đầu):")
        print(cleaned_text)
        
        # Áp dụng POS Tagging và Stemming
        pos_result = apply_stemming_and_pos(cleaned_text)
        
        if pos_result:
            print(f"\n🔤 TOKENIZATION:")
            print(f"Số từ sau tách: {len(pos_result['original_tokens'])}")
            print(f"10 từ đầu: {pos_result['original_tokens'][:10]}")
            
            print(f"\n🏷️  POS TAGGING (sử dụng HMM Model):")
            print(f"{'STT':<5} {'Từ':<30} {'POS Tag':<10}")
            print("-" * 50)
            for i, (word, pos) in enumerate(pos_result['word_pos_pairs'][:20], 1):
                print(f"{i:<5} {word:<30} {pos:<10}")
            
            print(f"\n🌱 STEMMING (Trích xuất từ quan trọng):")
            print(f"Số từ sau stemming: {len(pos_result['stemmed_tokens'])}")
            print(f"Từ đã stemming: {pos_result['stemmed_tokens'][:30]}")
            
            # Thống kê POS tags
            from collections import Counter
            pos_counter = Counter(pos_result['pos_tags'])
            print(f"\n📊 THỐNG KÊ POS TAGS:")
            for pos_tag, count in pos_counter.most_common(10):
                print(f"  {pos_tag}: {count} từ")
            
            results.append({
                'doc_id': str(doc.get('_id')),
                'title': doc.get('title', ''),
                'basic_result': basic_result,
                'pos_result': pos_result,
                'pos_statistics': dict(pos_counter)
            })
    
    return results

print("✓ analyze_document_with_pos function đã được định nghĩa")

✓ analyze_document_with_pos function đã được định nghĩa


### Demo: Chạy phân tích POS Tagging và Stemming

Chạy cell bên dưới để xem demo với 1 document ngẫu nhiên:

In [68]:
# Demo: Phân tích 1 document với POS Tagging và Stemming

# Kiểm tra kết nối database
try:
    collection.count_documents({})
    print("✓ Đã có kết nối database")
except:
    print("Đang kết nối database...")
    client, db, collection = connect_to_database()

# Kiểm tra collection có tồn tại không
if collection is None:
    print("❌ Lỗi: Không thể kết nối database. Vui lòng kiểm tra MONGO_URI!")
else:
    # Phân tích document
    results = analyze_document_with_pos(collection, limit=1)

✓ Đã có kết nối database

📄 PHÂN TÍCH DOCUMENT
Title: 'Ảo tưởng bóng đá Việt Nam vươn tầm khi giành vé dự VCK U23 châu Á'
URL: https://vnexpress.net/u23-viet-nam-u23-yemen-u23-viet-nam-thuong-chau-ao-tuong-bong-da-viet-nam-vuon-tam-khi-gianh-ve-du-vck-u23-chau-a-4937349.html
Date: Thứ tư, 10/9/2025, 16:30 (GMT+7)

📝 Text mẫu (1000 ký tự đầu):
ảo tưởng bóng đá việt nam vươn tầm khi giành vé dự vck u23 châu á u23 việt nam vừa giành vé dự vòng chung kết u23 châu á 2026 sau trận thắng 1 0 trước yemen đánh dấu lần thứ sáu liên tiếp góp mặt ở đấu trường châu lục này người hâm mộ vỡ òa truyền thông rộn ràng với những lời tung hô chiến thắng xứng đáng tấm vé ngọt ngào hay khẳng định vị thế nhưng khi nhìn lại một cách tỉnh táo tôi cho rằng sự hiện diện liên tục của u23 việt nam ở vck châu á không phản ánh một nền bóng đá đang lớn mạnh đó nhiều phần là nhờ may mắn trước hết cần nói rõ việc giành vé dự vck u23 châu á không đồng nghĩa với việc đội tuyển đã đạt tầm châu lục thể thức vòng loại hiện 

### Giải thích các POS Tags phổ biến trong tiếng Việt:

- **N** (Noun): Danh từ - VD: "bóng_đá", "cầu_thủ", "trận_đấu"
- **V** (Verb): Động từ - VD: "ghi", "chơi", "thi_đấu"
- **A** (Adjective): Tính từ - VD: "tốt", "mạnh", "xuất_sắc"
- **P** (Pronoun): Đại từ - VD: "họ", "chúng_tôi", "anh_ấy"
- **R** (Adverb): Phó từ - VD: "rất", "đã", "sẽ"
- **L** (Determiner): Hạn định từ - VD: "những", "các", "mỗi"
- **M** (Numeral): Số từ - VD: "một", "hai", "100"
- **E** (Preposition): Giới từ - VD: "ở", "tại", "trong"
- **C** (Conjunction): Liên từ - VD: "và", "hoặc", "nhưng"
- **I** (Interjection): Thán từ - VD: "ồ", "à", "ôi"

**HMM (Hidden Markov Model)** trong PyVi được sử dụng để dự đoán POS tags dựa trên:
1. Xác suất xuất hiện của từ với tag (Emission probability)
2. Xác suất chuyển đổi giữa các tags (Transition probability)

### Quy trình xử lý văn bản đầy đủ:

**Các bước xử lý text theo thứ tự:**

1. **Cleaning** - Làm sạch text (loại ký tự đặc biệt, chuẩn hóa)
2. **Tokenization** - Tách từ (dùng ViTokenizer)
3. **Stopwords Removal** - Loại bỏ từ dừng (các từ không mang ý nghĩa như "và", "của", "trong"...)
4. **POS Tagging** - Gắn nhãn từ loại (dùng HMM)
5. **Stemming/Lemmatization** - Chuẩn hóa từ về dạng gốc

## 6.6. Text Preprocessing Nâng Cao

### Các bước bổ sung:
1. **Normalization** - Chuẩn hóa số, ngày tháng, từ viết tắt
2. **Spell Correction** - Sửa lỗi chính tả
3. **Advanced Word Segmentation** - Xử lý từ ghép phức tạp

In [69]:
class AdvancedTextNormalizer:
    """Class xử lý chuẩn hóa văn bản nâng cao"""
    
    def __init__(self):
        # Chuẩn hóa số tiếng Việt
        self.number_mappings = {
            'không': '0', 'một': '1', 'hai': '2', 'ba': '3', 'bốn': '4',
            'năm': '5', 'sáu': '6', 'bảy': '7', 'tám': '8', 'chín': '9',
            'mười': '10', 'mươi': '0', 'trăm': '00', 'nghìn': '000',
            'triệu': '000000', 'tỷ': '000000000'
        }
        
        # Từ viết tắt phổ biến trong bóng đá
        self.abbreviations = {
            'hlv': 'huấn_luyện_viên',
            'gđ': 'giám_đốc',
            'clb': 'câu_lạc_bộ',
            'vff': 'liên_đoàn_bóng_đá_việt_nam',
            'aff': 'asean_football_federation',
            'uefa': 'union_of_european_football_associations',
            'fifa': 'federation_internationale_de_football_association',
            'vtv': 'đài_truyền_hình_việt_nam',
            'u23': 'under_23',
            'u22': 'under_22',
            'u19': 'under_19',
            'var': 'video_assistant_referee',
            'hvl': 'huấn_luyện_viên',
            'đtqg': 'đội_tuyển_quốc_gia',
            'slna': 'sông_lam_nghệ_an',
            'hagl': 'hoàng_anh_gia_lai',
            'tphcm': 'thành_phố_hồ_chí_minh'
        }
        
        # Pattern ngày tháng
        self.date_patterns = [
            r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})',  # 01/12/2024
            r'ngày\s+(\d{1,2})\s+tháng\s+(\d{1,2})\s+năm\s+(\d{4})',  # ngày 1 tháng 12 năm 2024
            r'(\d{1,2})\s*[/-]\s*(\d{1,2})\s*[/-]\s*(\d{2,4})',  # 1-12-24
        ]
    
    def normalize_numbers(self, text):
        """Chuẩn hóa số từ chữ sang số"""
        text_lower = text.lower()
        
        # Chuẩn hóa số đơn giản
        for word, digit in self.number_mappings.items():
            text_lower = text_lower.replace(f' {word} ', f' {digit} ')
        
        # Xử lý số phức tạp (VD: "hai mươi ba" -> "23")
        import re
        
        # Pattern: [số chục] mươi [số đơn vị]
        pattern = r'(\w+)\s+mươi\s+(\w+)'
        def replace_complex_number(match):
            tens = self.number_mappings.get(match.group(1), match.group(1))
            ones = self.number_mappings.get(match.group(2), match.group(2))
            if tens.isdigit() and ones.isdigit():
                return str(int(tens) * 10 + int(ones))
            return match.group(0)
        
        text_lower = re.sub(pattern, replace_complex_number, text_lower)
        
        return text_lower
    
    def expand_abbreviations(self, text):
        """Mở rộng từ viết tắt"""
        text_lower = text.lower()
        
        for abbr, full in self.abbreviations.items():
            # Thay thế từ viết tắt (với word boundary)
            pattern = r'\b' + abbr + r'\b'
            text_lower = re.sub(pattern, full, text_lower)
        
        return text_lower
    
    def normalize_dates(self, text):
        """Chuẩn hóa ngày tháng về format chuẩn YYYY-MM-DD"""
        import re
        
        # Pattern 1: dd/mm/yyyy hoặc dd-mm-yyyy
        pattern1 = r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})'
        text = re.sub(pattern1, r'\3-\2-\1', text)
        
        # Pattern 2: ngày X tháng Y năm Z
        pattern2 = r'ngày\s+(\d{1,2})\s+tháng\s+(\d{1,2})\s+năm\s+(\d{4})'
        text = re.sub(pattern2, r'\3-\2-\1', text)
        
        return text
    
    def normalize_all(self, text):
        """Áp dụng tất cả các chuẩn hóa"""
        text = self.normalize_dates(text)
        text = self.expand_abbreviations(text)
        text = self.normalize_numbers(text)
        return text

print("✓ AdvancedTextNormalizer class đã được định nghĩa")

✓ AdvancedTextNormalizer class đã được định nghĩa


In [70]:
class VietnameseSpellChecker:
    """Class kiểm tra và sửa lỗi chính tả tiếng Việt"""
    
    def __init__(self, vocab_file=None):
        # Load vocabulary từ file hoặc tạo từ corpus
        self.vocabulary = set()
        
        if vocab_file and os.path.exists(vocab_file):
            with open(vocab_file, 'r', encoding='utf-8') as f:
                self.vocabulary = set(line.strip().lower() for line in f)
            print(f"✓ Đã load {len(self.vocabulary)} từ từ vocab file")
        else:
            print("⚠️ Chưa có vocab file, sẽ tạo từ corpus")
    
    def build_vocabulary_from_collection(self, collection, limit=1000):
        """Xây dựng vocabulary từ MongoDB collection"""
        processor = VietnameseTextProcessor()
        docs = collection.find().limit(limit)
        
        print(f"Đang xây dựng vocabulary từ {limit} documents...")
        for doc in tqdm(docs, desc="Building vocab", total=limit):
            result = processor.process_document(doc)
            tokens = result['filtered_tokens']
            self.vocabulary.update(tokens)
        
        print(f"✓ Vocabulary có {len(self.vocabulary)} từ duy nhất")
        return self.vocabulary
    
    def save_vocabulary(self, filename="vocab.txt"):
        """Lưu vocabulary ra file"""
        filepath = os.path.join(os.getcwd(), filename)
        with open(filepath, 'w', encoding='utf-8') as f:
            for word in sorted(self.vocabulary):
                f.write(f"{word}\n")
        print(f"✓ Đã lưu vocabulary vào {filepath}")
    
    def edit_distance(self, s1, s2):
        """Tính Levenshtein distance giữa 2 chuỗi"""
        if len(s1) < len(s2):
            return self.edit_distance(s2, s1)
        
        if len(s2) == 0:
            return len(s1)
        
        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row
        
        return previous_row[-1]
    
    def suggest_corrections(self, word, max_distance=2, max_suggestions=5):
        """Gợi ý từ sửa lỗi chính tả"""
        word_lower = word.lower()
        
        # Nếu từ đã đúng
        if word_lower in self.vocabulary:
            return [word_lower]
        
        # Tìm từ gần nhất
        suggestions = []
        for vocab_word in self.vocabulary:
            distance = self.edit_distance(word_lower, vocab_word)
            if distance <= max_distance:
                suggestions.append((vocab_word, distance))
        
        # Sắp xếp theo khoảng cách
        suggestions.sort(key=lambda x: x[1])
        
        return [word for word, _ in suggestions[:max_suggestions]]
    
    def correct_text(self, text, auto_correct=False):
        """Kiểm tra và sửa lỗi chính tả cho text"""
        tokens = text.lower().split()
        corrected_tokens = []
        errors_found = []
        
        for token in tokens:
            if token in self.vocabulary:
                corrected_tokens.append(token)
            else:
                suggestions = self.suggest_corrections(token, max_distance=2, max_suggestions=3)
                
                if suggestions:
                    if auto_correct:
                        # Tự động chọn từ đầu tiên
                        corrected_tokens.append(suggestions[0])
                        errors_found.append({
                            'original': token,
                            'corrected': suggestions[0],
                            'suggestions': suggestions
                        })
                    else:
                        corrected_tokens.append(token)
                        errors_found.append({
                            'original': token,
                            'suggestions': suggestions
                        })
                else:
                    corrected_tokens.append(token)
        
        return {
            'original': text,
            'corrected': ' '.join(corrected_tokens),
            'errors': errors_found
        }

print("✓ VietnameseSpellChecker class đã được định nghĩa")

✓ VietnameseSpellChecker class đã được định nghĩa


### Demo: Chuẩn hóa nâng cao

Test các tính năng normalization:

In [71]:
# Demo: Chuẩn hóa nâng cao
normalizer = AdvancedTextNormalizer()

# Test cases
test_texts = [
    "HLV Park Hang-seo dẫn dắt đội U23 Việt Nam",
    "Trận đấu diễn ra vào ngày 15 tháng 10 năm 2024",
    "CLB HAGL thắng SLNA với tỷ số ba không",
    "Cầu thủ ghi được hai mươi ba bàn thắng",
    "VFF công bố danh sách đội tuyển vào 01/12/2024"
]

print("🔄 DEMO CHUẨN HÓA NÂNG CAO")
print("="*80)

for i, text in enumerate(test_texts, 1):
    print(f"\n{i}. Text gốc:")
    print(f"   {text}")
    
    # Chuẩn hóa từ viết tắt
    abbr_normalized = normalizer.expand_abbreviations(text)
    print(f"   Sau expand abbreviations:")
    print(f"   {abbr_normalized}")
    
    # Chuẩn hóa ngày tháng
    date_normalized = normalizer.normalize_dates(abbr_normalized)
    print(f"   Sau normalize dates:")
    print(f"   {date_normalized}")
    
    # Chuẩn hóa số
    number_normalized = normalizer.normalize_numbers(date_normalized)
    print(f"   Sau normalize numbers:")
    print(f"   {number_normalized}")
    
    print("-"*80)

🔄 DEMO CHUẨN HÓA NÂNG CAO

1. Text gốc:
   HLV Park Hang-seo dẫn dắt đội U23 Việt Nam
   Sau expand abbreviations:
   huấn_luyện_viên park hang-seo dẫn dắt đội under_23 việt nam
   Sau normalize dates:
   huấn_luyện_viên park hang-seo dẫn dắt đội under_23 việt nam
   Sau normalize numbers:
   huấn_luyện_viên park hang-seo dẫn dắt đội under_23 việt nam
--------------------------------------------------------------------------------

2. Text gốc:
   Trận đấu diễn ra vào ngày 15 tháng 10 năm 2024
   Sau expand abbreviations:
   trận đấu diễn ra vào ngày 15 tháng 10 năm 2024
   Sau normalize dates:
   trận đấu diễn ra vào 2024-10-15
   Sau normalize numbers:
   trận đấu diễn ra vào 2024-10-15
--------------------------------------------------------------------------------

3. Text gốc:
   CLB HAGL thắng SLNA với tỷ số ba không
   Sau expand abbreviations:
   câu_lạc_bộ hoàng_anh_gia_lai thắng sông_lam_nghệ_an với tỷ số ba không
   Sau normalize dates:
   câu_lạc_bộ hoàng_anh_gia_lai thắng 

### Demo: Spell Checker

Xây dựng vocabulary và kiểm tra lỗi chính tả:

In [72]:
# Demo: Spell Checker

# Kiểm tra xem đã có vocab file chưa
vocab_file = "vocab.txt"

if os.path.exists(vocab_file):
    print(f"✓ Đã tìm thấy file {vocab_file}")
    spell_checker = VietnameseSpellChecker(vocab_file)
else:
    print(f"⚠️ Chưa có file {vocab_file}, đang tạo vocabulary từ database...")
    
    # Kết nối database
    try:
        collection.count_documents({})
        print("✓ Đã có kết nối database")
    except:
        print("Đang kết nối database...")
        client, db, collection = connect_to_database()
    
    if collection is not None:
        # Tạo spell checker và build vocabulary
        spell_checker = VietnameseSpellChecker()
        spell_checker.build_vocabulary_from_collection(collection, limit=1000)
        spell_checker.save_vocabulary(vocab_file)
    else:
        print("❌ Không thể kết nối database để build vocabulary!")
        spell_checker = None

# Test spell checker
if spell_checker and len(spell_checker.vocabulary) > 0:
    print("\n" + "="*80)
    print("🔍 DEMO SPELL CHECKER")
    print("="*80)
    
    # Test cases với lỗi chính tả
    test_cases = [
        "bóng đá việt nam",
        "cầu thủ thi đấu",
        "huấn luyện viên", 
        "trận đấu quan trọng"
    ]
    
    for text in test_cases:
        print(f"\n📝 Text: {text}")
        result = spell_checker.correct_text(text, auto_correct=False)
        
        if result['errors']:
            print(f"⚠️ Tìm thấy {len(result['errors'])} lỗi tiềm năng:")
            for error in result['errors']:
                print(f"   - '{error['original']}' → Gợi ý: {error['suggestions']}")
        else:
            print(f"✓ Không có lỗi chính tả")
else:
    print("⚠️ Spell checker chưa sẵn sàng")

✓ Đã tìm thấy file vocab.txt
✓ Đã load 171 từ từ vocab file

🔍 DEMO SPELL CHECKER

📝 Text: bóng đá việt nam
⚠️ Tìm thấy 1 lỗi tiềm năng:
   - 'đá' → Gợi ý: ['']

📝 Text: cầu thủ thi đấu
✓ Không có lỗi chính tả

📝 Text: huấn luyện viên
✓ Không có lỗi chính tả

📝 Text: trận đấu quan trọng
✓ Không có lỗi chính tả


### Quy trình xử lý văn bản HOÀN CHỈNH (với các bước nâng cao)

Kết hợp tất cả các bước cơ bản + nâng cao:

In [73]:
def complete_advanced_pipeline(text, processor, normalizer=None, spell_checker=None):
    """
    Quy trình xử lý văn bản HOÀN CHỈNH:
    
    BƯỚC CƠ BẢN:
    1. Normalization (nâng cao)
    2. Spell Correction (optional)
    3. Cleaning
    4. Tokenization
    5. Stopwords Removal
    6. POS Tagging (HMM)
    7. Stemming
    """
    print("🔄 QUY TRÌNH XỬ LÝ VĂN BẢN HOÀN CHỈNH (CƠ BẢN + NÂNG CAO)")
    print("="*80)
    
    original_text = text
    
    # ============= PHẦN NÂNG CAO =============
    
    # Bước 0.1: ADVANCED NORMALIZATION
    if normalizer:
        print("\n📍 BƯỚC 0.1: ADVANCED NORMALIZATION")
        print("-"*80)
        print(f"Text gốc: {text[:200]}...")
        
        text = normalizer.normalize_all(text)
        print(f"Sau normalization: {text[:200]}...")
    
    # Bước 0.2: SPELL CORRECTION (optional)
    if spell_checker and len(spell_checker.vocabulary) > 0:
        print("\n📍 BƯỚC 0.2: SPELL CORRECTION")
        print("-"*80)
        
        result = spell_checker.correct_text(text, auto_correct=True)
        if result['errors']:
            print(f"Tìm thấy và sửa {len(result['errors'])} lỗi:")
            for error in result['errors'][:5]:  # Hiển thị 5 lỗi đầu
                print(f"  '{error['original']}' → '{error['corrected']}'")
            text = result['corrected']
        else:
            print("✓ Không tìm thấy lỗi chính tả")
    
    # ============= PHẦN CƠ BẢN =============
    
    # Bước 1: CLEANING
    print("\n📍 BƯỚC 1: CLEANING")
    print("-"*80)
    cleaned = processor.clean_text(text)
    print(f"Sau cleaning: {cleaned[:200]}...")
    
    # Bước 2: TOKENIZATION
    print("\n📍 BƯỚC 2: TOKENIZATION")
    print("-"*80)
    tokens = processor.tokenize_vietnamese(cleaned)
    print(f"Số tokens: {len(tokens)}")
    print(f"20 tokens đầu: {tokens[:20]}")
    
    # Bước 3: STOPWORDS REMOVAL
    print("\n📍 BƯỚC 3: STOPWORDS REMOVAL")
    print("-"*80)
    filtered_tokens = processor.remove_stopwords(tokens)
    removed = len(tokens) - len(filtered_tokens)
    print(f"Loại bỏ {removed} stopwords ({removed/len(tokens)*100:.1f}%)")
    print(f"Còn lại: {len(filtered_tokens)} tokens")
    
    # Bước 4: POS TAGGING (HMM)
    print("\n📍 BƯỚC 4: POS TAGGING - HMM")
    print("-"*80)
    if PYVI_AVAILABLE:
        pos_result = ViPosTagger.postagging(ViTokenizer.tokenize(cleaned))
        words, pos_tags = pos_result[0], pos_result[1]
        
        from collections import Counter
        pos_counter = Counter(pos_tags)
        print(f"Top 5 POS tags:")
        for tag, count in pos_counter.most_common(5):
            print(f"  {tag}: {count}")
    else:
        words, pos_tags = tokens, []
    
    # Bước 5: STEMMING
    print("\n📍 BƯỚC 5: STEMMING")
    print("-"*80)
    if PYVI_AVAILABLE and len(pos_tags) > 0:
        important_pos = ['N', 'V', 'A', 'M']
        stemmed = [w for w, p in zip(words, pos_tags) if any(p.startswith(x) for x in important_pos)]
        print(f"Giữ lại: {len(stemmed)}/{len(words)} từ quan trọng")
    else:
        stemmed = filtered_tokens
    
    # KẾT QUẢ
    print("\n" + "="*80)
    print("✅ HOÀN THÀNH")
    print("="*80)
    print(f"📊 Thống kê:")
    print(f"  • Text gốc: {len(original_text)} ký tự")
    print(f"  • Sau normalization: {len(text)} ký tự")
    print(f"  • Tokens: {len(tokens)}")
    print(f"  • Sau filter: {len(filtered_tokens)}")
    print(f"  • Sau stemming: {len(stemmed)}")
    print(f"  • Giảm: {(1 - len(stemmed)/len(tokens))*100:.1f}%")
    
    return {
        'original': original_text,
        'normalized': text,
        'cleaned': cleaned,
        'tokens': tokens,
        'filtered_tokens': filtered_tokens,
        'stemmed_tokens': stemmed,
        'pos_tags': pos_tags if PYVI_AVAILABLE else []
    }

print("✓ complete_advanced_pipeline function đã được định nghĩa")

✓ complete_advanced_pipeline function đã được định nghĩa


### Demo: Chạy quy trình HOÀN CHỈNH

Test quy trình với tất cả các bước nâng cao:

In [74]:
# Demo: Quy trình HOÀN CHỈNH

# Khởi tạo các components
try:
    processor
    print("✓ Đã có processor")
except:
    processor = VietnameseTextProcessor()
    print("✓ Đã khởi tạo processor")

try:
    normalizer
    print("✓ Đã có normalizer")
except:
    normalizer = AdvancedTextNormalizer()
    print("✓ Đã khởi tạo normalizer")

# Spell checker (optional)
try:
    spell_checker
    if spell_checker and len(spell_checker.vocabulary) > 0:
        print(f"✓ Đã có spell_checker với {len(spell_checker.vocabulary)} từ")
    else:
        print("⚠️ Spell checker chưa có vocabulary, sẽ bỏ qua bước này")
        spell_checker = None
except:
    print("⚠️ Spell checker chưa khởi tạo, sẽ bỏ qua bước này")
    spell_checker = None

# Test text
test_text = """
HLV Park Hang-seo đã dẫn dắt đội tuyển U23 Việt Nam 
thi đấu rất xuất sắc tại giải đấu ngày 15 tháng 1 năm 2024.
CLB HAGL đã thắng SLNA với tỷ số ba không trong trận đấu vòng năm V-League.
Cầu thủ Nguyễn Quang Hải ghi được hai mươi ba bàn thắng trong mùa giải này.
"""

print("\n📄 Test Text:")
print(test_text)
print("\n")

# Chạy quy trình hoàn chỉnh
result = complete_advanced_pipeline(
    test_text[:500],  # Giới hạn để demo
    processor=processor,
    normalizer=normalizer,
    spell_checker=spell_checker
)

✓ Đã có processor
✓ Đã có normalizer
✓ Đã có spell_checker với 171 từ

📄 Test Text:

HLV Park Hang-seo đã dẫn dắt đội tuyển U23 Việt Nam 
thi đấu rất xuất sắc tại giải đấu ngày 15 tháng 1 năm 2024.
CLB HAGL đã thắng SLNA với tỷ số ba không trong trận đấu vòng năm V-League.
Cầu thủ Nguyễn Quang Hải ghi được hai mươi ba bàn thắng trong mùa giải này.



🔄 QUY TRÌNH XỬ LÝ VĂN BẢN HOÀN CHỈNH (CƠ BẢN + NÂNG CAO)

📍 BƯỚC 0.1: ADVANCED NORMALIZATION
--------------------------------------------------------------------------------
Text gốc: 
HLV Park Hang-seo đã dẫn dắt đội tuyển U23 Việt Nam 
thi đấu rất xuất sắc tại giải đấu ngày 15 tháng 1 năm 2024.
CLB HAGL đã thắng SLNA với tỷ số ba không trong trận đấu vòng năm V-League.
Cầu thủ Ng...
Sau normalization: 
huấn_luyện_viên park hang-seo đã dẫn dắt đội tuyển under_23 việt nam 
thi đấu rất xuất sắc tại giải đấu 2024-1-15.
câu_lạc_bộ hoàng_anh_gia_lai đã thắng sông_lam_nghệ_an với 000000000 số 3 0 trong tr...

📍 BƯỚC 0.2: SPELL CORRECTION
------

### Lưu kết quả preprocessing vào file

Các function để lưu output:

In [ ]:
def save_preprocessing_results(results, output_dir="d:\\data\\Search_Engine\\outputs", prefix="preprocessing"):
    """
    Lưu kết quả preprocessing vào các file khác nhau
    
    Args:
        results: dict chứa kết quả preprocessing
        output_dir: thư mục lưu output
        prefix: tiền tố tên file
    """
    import os
    from datetime import datetime
    
    # Tạo folder outputs
    os.makedirs(output_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. Lưu summary dạng JSON
    summary = {
        'timestamp': timestamp,
        'original_length': len(results['original']),
        'normalized_length': len(results.get('normalized', results['original'])),
        'cleaned_length': len(results['cleaned']),
        'token_count': len(results['tokens']),
        'filtered_count': len(results['filtered_tokens']),
        'stemmed_count': len(results['stemmed_tokens']),
        'reduction_rate': f"{(1 - len(results['stemmed_tokens'])/len(results['tokens']))*100:.1f}%"
    }
    
    summary_file = os.path.join(output_dir, f"{prefix}_summary_{timestamp}.json")
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    print(f"✓ Đã lưu summary vào: {summary_file}")
    
    # 2. Lưu tokens vào text file
    tokens_file = os.path.join(output_dir, f"{prefix}_tokens_{timestamp}.txt")
    with open(tokens_file, 'w', encoding='utf-8') as f:
        f.write("=== ALL TOKENS ===\n")
        f.write(' '.join(results['tokens']))
        f.write("\n\n=== FILTERED TOKENS (sau stopwords removal) ===\n")
        f.write(' '.join(results['filtered_tokens']))
        f.write("\n\n=== STEMMED TOKENS ===\n")
        f.write(' '.join(results['stemmed_tokens']))
    print(f"✓ Đã lưu tokens vào: {tokens_file}")
    
    # 3. Lưu POS tags nếu có
    if results.get('pos_tags') and len(results['pos_tags']) > 0:
        pos_file = os.path.join(output_dir, f"{prefix}_pos_tags_{timestamp}.csv")
        
        # Tạo dataframe với từ và POS tag
        if 'normalized' in results:
            # Nếu có normalized, cần tokenize lại
            if PYVI_AVAILABLE:
                from pyvi import ViTokenizer
                tokens_for_pos = ViTokenizer.tokenize(results['cleaned']).split()
            else:
                tokens_for_pos = results['tokens']
        else:
            tokens_for_pos = results['tokens']
        
        # Tạo DataFrame
        pos_data = []
        for i, (word, pos) in enumerate(zip(tokens_for_pos, results['pos_tags']), 1):
            pos_data.append({
                'Index': i,
                'Word': word,
                'POS_Tag': pos
            })
        
        df_pos = pd.DataFrame(pos_data)
        df_pos.to_csv(pos_file, index=False, encoding='utf-8-sig')
        print(f"✓ Đã lưu POS tags vào: {pos_file}")
    
    # 4. Lưu text đã xử lý từng bước
    steps_file = os.path.join(output_dir, f"{prefix}_steps_{timestamp}.txt")
    with open(steps_file, 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("TEXT PREPROCESSING - TỪNG BƯỚC\n")
        f.write("="*80 + "\n\n")
        
        f.write("1. TEXT GỐC:\n")
        f.write("-"*80 + "\n")
        f.write(results['original'][:500] + "...\n\n")
        
        if 'normalized' in results:
            f.write("2. SAU NORMALIZATION:\n")
            f.write("-"*80 + "\n")
            f.write(results['normalized'][:500] + "...\n\n")
        
        f.write("3. SAU CLEANING:\n")
        f.write("-"*80 + "\n")
        f.write(results['cleaned'][:500] + "...\n\n")
        
        f.write("4. SAU TOKENIZATION:\n")
        f.write("-"*80 + "\n")
        f.write(' '.join(results['tokens'][:100]) + "...\n\n")
        
        f.write("5. SAU STOPWORDS REMOVAL:\n")
        f.write("-"*80 + "\n")
        f.write(' '.join(results['filtered_tokens'][:100]) + "...\n\n")
        
        f.write("6. SAU STEMMING:\n")
        f.write("-"*80 + "\n")
        f.write(' '.join(results['stemmed_tokens'][:100]) + "...\n\n")
    
    print(f"✓ Đã lưu các bước xử lý vào: {steps_file}")
    
    print(f"\n📁 Tất cả output đã được lưu vào folder: {output_dir}")
    
    return {
        'summary_file': summary_file,
        'tokens_file': tokens_file,
        'pos_file': pos_file if results.get('pos_tags') else None,
        'steps_file': steps_file
    }

print("✓ save_preprocessing_results function đã được định nghĩa")

### Demo: Lưu kết quả preprocessing

Chạy quy trình và lưu output vào folder:

In [ ]:
# Demo: Lưu kết quả preprocessing

# Kiểm tra các components
try:
    collection.count_documents({})
    print("✓ Đã có kết nối database")
except:
    print("Đang kết nối database...")
    client, db, collection = connect_to_database()

try:
    processor
except:
    processor = VietnameseTextProcessor()

try:
    normalizer
except:
    normalizer = AdvancedTextNormalizer()

# Spell checker (optional)
try:
    spell_checker
    if not spell_checker or len(spell_checker.vocabulary) == 0:
        spell_checker = None
except:
    spell_checker = None

if collection is not None:
    # Lấy 1 document
    doc = collection.find_one()
    if doc:
        full_text = f"{doc.get('title', '')} {doc.get('content', '')}"
        
        print(f"\n📄 Processing document: {doc.get('title', 'N/A')[:100]}...\n")
        
        # Chạy quy trình hoàn chỉnh
        result = complete_advanced_pipeline(
            full_text[:2000],
            processor=processor,
            normalizer=normalizer,
            spell_checker=spell_checker
        )
        
        # Lưu kết quả vào file
        print("\n" + "="*80)
        print("💾 ĐANG LƯU KẾT QUẢ VÀO FILE...")
        print("="*80)
        
        saved_files = save_preprocessing_results(
            result,
            output_dir="d:\\data\\Search_Engine\\outputs",
            prefix="advanced_preprocessing"
        )
        
        print(f"\n✅ Đã hoàn thành và lưu tất cả output!")
    else:
        print("Không tìm thấy document!")
else:
    print("❌ Không thể kết nối database!")

### Batch Processing: Xử lý nhiều documents và lưu kết quả

Xử lý hàng loạt documents và lưu thống kê tổng hợp:

In [ ]:
def batch_process_and_save(collection, processor, limit=100, normalizer=None, output_dir="d:\\data\\Search_Engine\\outputs"):
    """
    Xử lý nhiều documents và lưu kết quả tổng hợp
    """
    from datetime import datetime
    
    print(f"🔄 BẮT ĐẦU XỬ LÝ BATCH: {limit} DOCUMENTS")
    print("="*80)
    
    # Tạo folder
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Lấy documents
    docs = collection.find().limit(limit)
    
    all_results = []
    stats = {
        'total_docs': 0,
        'total_tokens': 0,
        'total_filtered': 0,
        'total_stemmed': 0,
        'avg_reduction_rate': 0
    }
    
    print(f"\nĐang xử lý {limit} documents...")
    
    for doc in tqdm(docs, desc="Processing", total=limit):
        try:
            # Xử lý cơ bản
            result = processor.process_document(doc)
            
            # Áp dụng normalization nếu có
            if normalizer:
                cleaned = normalizer.normalize_all(result['cleaned_text'])
            else:
                cleaned = result['cleaned_text']
            
            # Thống kê
            stats['total_docs'] += 1
            stats['total_tokens'] += result['token_count']
            stats['total_filtered'] += result['filtered_count']
            stats['total_stemmed'] += result['filtered_count']  # Simplified
            
            all_results.append({
                'doc_id': result['doc_id'],
                'title': result['title'],
                'token_count': result['token_count'],
                'filtered_count': result['filtered_count'],
                'reduction_rate': (1 - result['filtered_count']/result['token_count'])*100 if result['token_count'] > 0 else 0
            })
            
        except Exception as e:
            print(f"Lỗi xử lý document: {e}")
    
    # Tính trung bình
    if stats['total_docs'] > 0:
        stats['avg_tokens_per_doc'] = stats['total_tokens'] / stats['total_docs']
        stats['avg_filtered_per_doc'] = stats['total_filtered'] / stats['total_docs']
        stats['avg_reduction_rate'] = (1 - stats['total_filtered']/stats['total_tokens'])*100 if stats['total_tokens'] > 0 else 0
    
    # Lưu thống kê tổng hợp
    stats_file = os.path.join(output_dir, f"batch_stats_{timestamp}.json")
    with open(stats_file, 'w', encoding='utf-8') as f:
        json.dump(stats, f, ensure_ascii=False, indent=2)
    print(f"\n✓ Đã lưu thống kê tổng hợp: {stats_file}")
    
    # Lưu chi tiết từng document
    details_file = os.path.join(output_dir, f"batch_details_{timestamp}.csv")
    df = pd.DataFrame(all_results)
    df.to_csv(details_file, index=False, encoding='utf-8-sig')
    print(f"✓ Đã lưu chi tiết documents: {details_file}")
    
    # In thống kê
    print("\n" + "="*80)
    print("📊 THỐNG KÊ TỔNG HỢP")
    print("="*80)
    print(f"Tổng số documents: {stats['total_docs']}")
    print(f"Tổng số tokens: {stats['total_tokens']:,}")
    print(f"Trung bình tokens/doc: {stats['avg_tokens_per_doc']:.1f}")
    print(f"Trung bình filtered/doc: {stats['avg_filtered_per_doc']:.1f}")
    print(f"Tỷ lệ giảm trung bình: {stats['avg_reduction_rate']:.1f}%")
    
    return {
        'stats': stats,
        'results': all_results,
        'stats_file': stats_file,
        'details_file': details_file
    }

print("✓ batch_process_and_save function đã được định nghĩa")

### Demo: Batch processing

Xử lý hàng loạt 100 documents (có thể thay đổi số lượng):

In [ ]:
# Demo: Batch Processing 100 documents

# Kiểm tra các components
try:
    collection.count_documents({})
    print("✓ Đã có kết nối database")
except:
    print("Đang kết nối database...")
    client, db, collection = connect_to_database()

try:
    processor
    print("✓ Đã có processor")
except:
    processor = VietnameseTextProcessor()

try:
    normalizer
    print("✓ Đã có normalizer")
except:
    normalizer = AdvancedTextNormalizer()

if collection is not None:
    # Chạy batch processing
    batch_results = batch_process_and_save(
        collection=collection,
        processor=processor,
        limit=100,  # Thay đổi số lượng documents tại đây
        normalizer=normalizer,
        output_dir="d:\\data\\Search_Engine\\outputs"
    )
    
    print(f"\n✅ Hoàn thành batch processing!")
    print(f"📁 Các file đã lưu:")
    print(f"   - Stats: {batch_results['stats_file']}")
    print(f"   - Details: {batch_results['details_file']}")
else:
    print("❌ Không thể kết nối database!")

In [75]:
def complete_text_processing_pipeline(text, processor):
    """
    Quy trình xử lý văn bản ĐẦY ĐỦ theo thứ tự:
    1. Cleaning
    2. Tokenization
    3. Stopwords Removal
    4. POS Tagging (HMM)
    5. Stemming
    """
    print("🔄 BẮT ĐẦU QUY TRÌNH XỬ LÝ VĂN BẢN")
    print("="*80)
    
    # Bước 1: CLEANING
    print("\n📍 BƯỚC 1: CLEANING (Làm sạch văn bản)")
    print("-"*80)
    cleaned = processor.clean_text(text)
    print(f"Text gốc: {text[:200]}...")
    print(f"Text sau cleaning: {cleaned[:200]}...")
    print(f"Độ dài: {len(text)} → {len(cleaned)} ký tự")
    
    # Bước 2: TOKENIZATION
    print("\n📍 BƯỚC 2: TOKENIZATION (Tách từ)")
    print("-"*80)
    tokens = processor.tokenize_vietnamese(cleaned)
    print(f"Số từ sau tách: {len(tokens)}")
    print(f"20 từ đầu tiên: {tokens[:20]}")
    
    # Bước 3: STOPWORDS REMOVAL
    print("\n📍 BƯỚC 3: STOPWORDS REMOVAL (Loại bỏ từ dừng)")
    print("-"*80)
    print(f"Danh sách stopwords ({len(processor.stop_words)} từ):")
    print(list(processor.stop_words)[:20], "...")
    
    filtered_tokens = processor.remove_stopwords(tokens)
    removed_count = len(tokens) - len(filtered_tokens)
    print(f"\nSố từ trước: {len(tokens)}")
    print(f"Số từ sau: {len(filtered_tokens)}")
    print(f"Đã loại bỏ: {removed_count} từ dừng ({removed_count/len(tokens)*100:.1f}%)")
    print(f"20 từ còn lại: {filtered_tokens[:20]}")
    
    # Bước 4: POS TAGGING (HMM)
    print("\n📍 BƯỚC 4: POS TAGGING - HMM (Gắn nhãn từ loại)")
    print("-"*80)
    if PYVI_AVAILABLE:
        # POS tagging trên text đã cleaned (chưa remove stopwords để đảm bảo ngữ cảnh)
        pos_result = ViPosTagger.postagging(ViTokenizer.tokenize(cleaned))
        words = pos_result[0]
        pos_tags = pos_result[1]
        
        print(f"{'STT':<5} {'Từ':<25} {'POS Tag':<10}")
        print("-"*45)
        for i, (word, pos) in enumerate(zip(words[:15], pos_tags[:15]), 1):
            print(f"{i:<5} {word:<25} {pos:<10}")
        
        # Thống kê POS
        from collections import Counter
        pos_counter = Counter(pos_tags)
        print(f"\n📊 Thống kê POS Tags:")
        for pos_tag, count in pos_counter.most_common(5):
            print(f"  {pos_tag}: {count} từ")
    else:
        print("⚠️ PyVi chưa cài đặt, bỏ qua bước POS Tagging")
        words, pos_tags = tokens, []
    
    # Bước 5: STEMMING
    print("\n📍 BƯỚC 5: STEMMING (Trích xuất từ quan trọng)")
    print("-"*80)
    if PYVI_AVAILABLE and len(pos_tags) > 0:
        important_pos = ['N', 'V', 'A', 'M']
        stemmed = [word for word, pos in zip(words, pos_tags) 
                   if any(pos.startswith(p) for p in important_pos)]
        print(f"Chỉ giữ lại: Danh từ (N), Động từ (V), Tính từ (A), Số từ (M)")
        print(f"Số từ trước stemming: {len(words)}")
        print(f"Số từ sau stemming: {len(stemmed)}")
        print(f"30 từ đã stemming: {stemmed[:30]}")
    else:
        stemmed = filtered_tokens
        print(f"Sử dụng filtered tokens: {len(stemmed)} từ")
    
    # Tổng kết
    print("\n" + "="*80)
    print("✅ HOÀN THÀNH QUY TRÌNH XỬ LÝ")
    print("="*80)
    print(f"📊 Tóm tắt:")
    print(f"  • Text gốc: {len(text)} ký tự, {len(text.split())} từ")
    print(f"  • Sau cleaning: {len(cleaned)} ký tự")
    print(f"  • Sau tokenization: {len(tokens)} tokens")
    print(f"  • Sau stopwords removal: {len(filtered_tokens)} tokens")
    print(f"  • Sau stemming: {len(stemmed)} tokens")
    print(f"  • Tỷ lệ giảm: {(1 - len(stemmed)/len(tokens))*100:.1f}%")
    
    return {
        'original': text,
        'cleaned': cleaned,
        'tokens': tokens,
        'filtered_tokens': filtered_tokens,
        'stemmed_tokens': stemmed,
        'pos_tags': pos_tags if PYVI_AVAILABLE else []
    }

print("✓ complete_text_processing_pipeline function đã được định nghĩa")

✓ complete_text_processing_pipeline function đã được định nghĩa


### Demo: Chạy quy trình xử lý văn bản ĐẦY ĐỦ

Chạy cell bên dưới để xem demo với 1 document từ database (bao gồm CẢ 5 BƯỚC):

In [76]:
# Demo: Quy trình xử lý văn bản ĐẦY ĐỦ

# Kiểm tra kết nối database
try:
    collection.count_documents({})
    print("✓ Đã có kết nối database")
except:
    print("Đang kết nối database...")
    client, db, collection = connect_to_database()

# Kiểm tra processor
try:
    processor
    print("✓ Đã có processor")
except:
    processor = VietnameseTextProcessor()
    print("✓ Đã khởi tạo processor")

# Kiểm tra collection có tồn tại không
if collection is None:
    print("❌ Lỗi: Không thể kết nối database. Vui lòng kiểm tra MONGO_URI!")
else:
    # Lấy 1 document mẫu
    doc = collection.find_one()
    if doc:
        full_text = f"{doc.get('title', '')} {doc.get('content', '')}"
        print(f"\n📄 Document: {doc.get('title', 'N/A')}\n")
        
        # Chạy quy trình đầy đủ
        result = complete_text_processing_pipeline(full_text[:2000], processor)
    else:
        print("Không tìm thấy document trong collection!")

✓ Đã có kết nối database
✓ Đã có processor

📄 Document: 'Ảo tưởng bóng đá Việt Nam vươn tầm khi giành vé dự VCK U23 châu Á'

🔄 BẮT ĐẦU QUY TRÌNH XỬ LÝ VĂN BẢN

📍 BƯỚC 1: CLEANING (Làm sạch văn bản)
--------------------------------------------------------------------------------
Text gốc: 'Ảo tưởng bóng đá Việt Nam vươn tầm khi giành vé dự VCK U23 châu Á' U23 Việt Nam vừa giành vé dự Vòng chung kết U23 châu Á 2026 sau trận thắng 1-0 trước Yemen, đánh dấu lần thứ sáu liên tiếp góp mặt ở...
Text sau cleaning: ảo tưởng bóng đá việt nam vươn tầm khi giành vé dự vck u23 châu á u23 việt nam vừa giành vé dự vòng chung kết u23 châu á 2026 sau trận thắng 1 0 trước yemen đánh dấu lần thứ sáu liên tiếp góp mặt ở đấ...
Độ dài: 2000 → 1936 ký tự

📍 BƯỚC 2: TOKENIZATION (Tách từ)
--------------------------------------------------------------------------------
Số từ sau tách: 353
20 từ đầu tiên: ['ảo_tưởng', 'bóng_đá', 'việt', 'nam', 'vươn', 'tầm', 'khi', 'giành', 'vé', 'dự', 'vck', 'u23', 'châu', 'á'

In [77]:
def analyze_multiple_documents(collection, processor, n=50, top_n=50, filename=None):
    """Lấy ngẫu nhiên n documents, gộp tokens và tính tần suất chung"""
    docs = collection.aggregate([{"$sample": {"size": n}}])
    all_tokens = []
    
    for idx, doc in enumerate(docs, 1):
        result = processor.process_document(doc)
        tokens = result["filtered_tokens"]
        all_tokens.extend(tokens)   # gộp tất cả tokens lại
    
    # Tính tần suất từ toàn bộ tokens
    counter = Counter(all_tokens)
    total = sum(counter.values())
    
    data = []
    for i, (word, freq) in enumerate(counter.most_common(top_n), 1):
        Pr = freq / total * 100
        data.append({
            "Word": word,
            "Freq": freq,
            "r": i,
            "Pr(%)": round(Pr, 2),
            "r*Pr": round(i * Pr / 100, 3)
        })
    
    df = pd.DataFrame(data)
    print(df.to_string(index=False))
    
    # Tạo folder outputs nếu chưa có
    output_dir = os.path.join("d:\\data\\Search_Engine", "outputs")
    os.makedirs(output_dir, exist_ok=True)
    
    # Xuất file CSV
    if not filename:
        filename = f"wordfreq_{n}docs_total_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    filepath = os.path.join(output_dir, filename)
    df.to_csv(filepath, index=False, encoding="utf-8-sig")
    print(f"\n✓ Đã lưu kết quả {n} documents (gộp) vào: {filepath}")
    
    return df

print("✓ analyze_multiple_documents function đã được định nghĩa")

✓ analyze_multiple_documents function đã được định nghĩa


## 8. N-grams Analysis

### Hàm phân tích N-grams

In [78]:
def generate_ngrams(tokens, n):
    """Sinh n-grams từ list tokens"""
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

print("✓ generate_ngrams function đã được định nghĩa")

✓ generate_ngrams function đã được định nghĩa


In [79]:
def analyze_ngrams_combined(collection, processor, n_docs=1800, max_n=5, top_k=50, filename=None):
    """
    Phân tích n-grams (1 -> max_n), lưu tất cả vào 1 file CSV duy nhất
    """
    docs = collection.aggregate([{"$sample": {"size": n_docs}}])

    all_tokens_list = []
    for doc in docs:
        result = processor.process_document(doc)
        all_tokens_list.append(result["filtered_tokens"])

    all_data = []

    for n in range(1, max_n+1):
        ngrams_all = []
        for tokens in all_tokens_list:
            ngrams_all.extend(generate_ngrams(tokens, n))
        
        counter = Counter(ngrams_all)
        for phrase, freq in counter.most_common(top_k):
            all_data.append({
                "n": n,
                "Frequency": freq,
                "Phrase": phrase
            })
    
    df = pd.DataFrame(all_data)

    # Tạo folder outputs nếu chưa có
    output_dir = os.path.join("d:\\data\\Search_Engine", "outputs")
    os.makedirs(output_dir, exist_ok=True)
    
    if not filename:
        filename = f"ngrams_combined_{n_docs}docs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    filepath = os.path.join(output_dir, filename)
    df.to_csv(filepath, index=False, encoding="utf-8-sig")
    print(f"✓ Đã lưu kết quả n-grams (1-{max_n}) vào: {filepath}")
    
    return df

print("✓ analyze_ngrams_combined function đã được định nghĩa")

✓ analyze_ngrams_combined function đã được định nghĩa


In [80]:
def analyze_single_ngram(collection, processor, n=1, n_docs=1800, top_k=100, filename=None):
    """
    Phân tích một loại n-gram cụ thể (1-gram, 2-gram, etc.)
    """
    docs = collection.aggregate([{"$sample": {"size": n_docs}}])

    all_tokens_list = []
    print(f"\nĐang xử lý {n_docs} documents cho {n}-gram...")
    for doc in tqdm(docs, desc=f"Processing {n}-gram", total=n_docs):
        result = processor.process_document(doc)
        all_tokens_list.append(result["filtered_tokens"])

    # Sinh n-grams
    ngrams_all = []
    for tokens in all_tokens_list:
        ngrams_all.extend(generate_ngrams(tokens, n))
    
    # Đếm tần suất
    counter = Counter(ngrams_all)
    total = sum(counter.values())
    
    data = []
    for i, (phrase, freq) in enumerate(counter.most_common(top_k), 1):
        Pr = freq / total * 100
        data.append({
            "Rank": i,
            "Phrase": phrase,
            "Frequency": freq,
            "Pr(%)": round(Pr, 2)
        })
    
    df = pd.DataFrame(data)
    print(f"\n{'='*80}")
    print(f"TOP {top_k} {n}-GRAM")
    print(f"{'='*80}")
    print(df.to_string(index=False))
    
    # Tạo folder outputs nếu chưa có
    output_dir = os.path.join("d:\\data\\Search_Engine", "outputs")
    os.makedirs(output_dir, exist_ok=True)
    
    # Lưu file
    if not filename:
        ngram_names = {1: "unigram", 2: "bigram", 3: "trigram", 4: "fourgram", 5: "fivegram"}
        name = ngram_names.get(n, f"{n}gram")
        filename = f"{name}_{n_docs}docs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    filepath = os.path.join(output_dir, filename)
    df.to_csv(filepath, index=False, encoding="utf-8-sig")
    print(f"\n✓ Đã lưu kết quả {n}-gram vào: {filepath}")
    
    return df

print("✓ analyze_single_ngram function đã được định nghĩa")

✓ analyze_single_ngram function đã được định nghĩa


---

## 8.1. Phân Tích Từng Loại N-gram

### Chạy các cell bên dưới để phân tích từng loại n-gram riêng biệt

### Bước 1: Kết nối Database và khởi tạo Processor

Nếu chưa kết nối, chạy cell này:

In [81]:
# Kết nối database và khởi tạo processor (nếu chưa làm)

# Kiểm tra kết nối database
try:
    collection.count_documents({})
    print("✓ Đã có kết nối database")
except:
    print("Đang kết nối database...")
    client, db, collection = connect_to_database()
    
    if collection is None:
        print("❌ Lỗi: Không thể kết nối database. Vui lòng kiểm tra MONGO_URI!")
    
# Kiểm tra processor
try:
    processor
    print("✓ Đã có processor")
except:
    print("Đang khởi tạo processor...")
    processor = VietnameseTextProcessor()
    print("✓ Processor đã sẵn sàng")

✓ Đã có kết nối database
✓ Đã có processor


### Bước 2: Phân tích 1-gram (Unigram)

Phân tích các từ đơn xuất hiện nhiều nhất:

In [82]:
# Phân tích 1-gram (Unigram)
df_unigram = analyze_single_ngram(collection, processor, n=1, n_docs=1830, top_k=100)


Đang xử lý 1830 documents cho 1-gram...


Processing 1-gram:   0%|          | 0/1830 [00:00<?, ?it/s]

Processing 1-gram: 100%|██████████| 1830/1830 [00:25<00:00, 71.56it/s] 




TOP 100 1-GRAM
 Rank    Phrase  Frequency  Pr(%)
    1       nam       8300   1.11
    2       đội       7740   1.04
    3      trận       7570   1.01
    4       hai       6793   0.91
    5      việt       6765   0.91
    6   cầu_thủ       6651   0.89
    7      bóng       5716   0.77
    8       hlv       5609   0.75
    9       năm       5198   0.70
   10    league       5168   0.69
   11     trước       4815   0.65
   12       sân       4734   0.63
   13       đến       4517   0.61
   14      phải       4336   0.58
   15       văn       3970   0.53
   16    nguyễn       3944   0.53
   17       bàn       3914   0.52
   18     thắng       3867   0.52
   19     nhiều       3816   0.51
   20      phút       3683   0.49
   21       anh       3654   0.49
   22      nhất       3537   0.47
   23      vòng       3494   0.47
   24       ông       3376   0.45
   25  việt_nam       3361   0.45
   26      giải       3264   0.44
   27        họ       3158   0.42
   28    hà_nội       3130   0.4

### Bước 3: Phân tích 2-gram (Bigram)

Phân tích các cặp từ liên tiếp xuất hiện nhiều nhất:

In [83]:
# Phân tích 2-gram (Bigram)
df_bigram = analyze_single_ngram(collection, processor, n=2, n_docs=1830, top_k=100)


Đang xử lý 1830 documents cho 2-gram...


Processing 2-gram: 100%|██████████| 1830/1830 [00:45<00:00, 40.11it/s] 




TOP 100 2-GRAM
 Rank           Phrase  Frequency  Pr(%)
    1         việt nam       5337   0.72
    2         đội bóng       2054   0.28
    3         thái lan       1886   0.25
    4         nam định       1768   0.24
    5           tp hcm       1318   0.18
    6        hải phòng       1078   0.14
    7       bình dương       1039   0.14
    8        hà_nội fc       1009   0.14
    9          chủ nhà        981   0.13
   10        bàn thắng        933   0.13
   11          hlv kim        874   0.12
   12        asean cup        815   0.11
   13         mùa giải        766   0.10
   14        bình định        762   0.10
   15          thứ hai        752   0.10
   16       hiếu lương        721   0.10
   17        quang hải        695   0.09
   18         vòng cấm        671   0.09
   19     bóng_đá việt        665   0.09
   20          aff cup        665   0.09
   21         hiệp hai        650   0.09
   22          hai đội        648   0.09
   23         sang sik        623   0.08


### Bước 4: Phân tích 3-gram (Trigram)

Phân tích các cụm 3 từ liên tiếp xuất hiện nhiều nhất:

In [84]:
# Phân tích 3-gram (Trigram)
df_trigram = analyze_single_ngram(collection, processor, n=3, n_docs=1830, top_k=100)


Đang xử lý 1830 documents cho 3-gram...


Processing 3-gram: 100%|██████████| 1830/1830 [00:52<00:00, 34.80it/s] 




TOP 100 3-GRAM
 Rank                     Phrase  Frequency  Pr(%)
    1               kim sang sik        617   0.08
    2           bóng_đá việt nam        615   0.08
    3               hlv kim sang        514   0.07
    4             asean cup 2024        465   0.06
    5              park hang seo        355   0.05
    6           cầu_thủ việt nam        347   0.05
    7               sân hàng đẫy        344   0.05
    8               ảnh đức đồng        295   0.04
    9           league 2024 2025        291   0.04
   10                 clb tp hcm        283   0.04
   11             ảnh hiếu lương        273   0.04
   12              hlv park hang        261   0.04
   13              bùi tiến dũng        259   0.03
   14                đội chủ nhà        247   0.03
   15         đội_tuyển việt nam        241   0.03
   16             tuyển việt nam        234   0.03
   17           nguyễn quang hải        231   0.03
   18               sân việt trì        224   0.03
   19          

### Bước 5: Phân tích 4-gram (Four-gram)

Phân tích các cụm 4 từ liên tiếp xuất hiện nhiều nhất:

In [85]:
# Phân tích 4-gram (Four-gram)
df_fourgram = analyze_single_ngram(collection, processor, n=4, n_docs=1830, top_k=100)


Đang xử lý 1830 documents cho 4-gram...


Processing 4-gram: 100%|██████████| 1830/1830 [01:00<00:00, 30.38it/s]



TOP 100 4-GRAM
 Rank                                Phrase  Frequency  Pr(%)
    1                      hlv kim sang sik        517   0.07
    2                     hlv park hang seo        259   0.03
    3              vòng_loại cuối asian cup        141   0.02
    4                    bùi hoàng việt anh        128   0.02
    5                   cuối asian cup 2027        121   0.02
    6                      đội bóng phố núi        112   0.02
    7                      hlv vũ hồng việt        105   0.01
    8                   hlv chu đình nghiêm        103   0.01
    9          vòng_loại hai world_cup 2026        102   0.01
   10                     hlv vũ tiến thành         98   0.01
   11        bóng_đá chuyên_nghiệp việt nam         92   0.01
   12                     mang dòng máu lai         89   0.01
   13                  league 2024 2025 sân         79   0.01
   14                      việt trì phú thọ         78   0.01
   15                   bảng asean cup 2024         75

### Bước 6: Phân tích 5-gram (Five-gram)

Phân tích các cụm 5 từ liên tiếp xuất hiện nhiều nhất:

In [86]:
# Phân tích 5-gram (Five-gram)
df_fivegram = analyze_single_ngram(collection, processor, n=5, n_docs=1830, top_k=100)


Đang xử lý 1830 documents cho 5-gram...


Processing 5-gram: 100%|██████████| 1830/1830 [00:27<00:00, 66.93it/s] 




TOP 100 5-GRAM
 Rank                                     Phrase  Frequency  Pr(%)
    1              vòng_loại cuối asian cup 2027        121   0.02
    2 công_ty cổ_phần bóng_đá chuyên_nghiệp việt         72   0.01
    3     cổ_phần bóng_đá chuyên_nghiệp việt nam         72   0.01
    4              bảng vòng_loại cuối asian cup         72   0.01
    5         bóng_đá chuyên_nghiệp việt nam vpf         69   0.01
    6                       sân việt trì phú thọ         66   0.01
    7                  cầu_thủ mang dòng máu lai         64   0.01
    8                      aff cup nay asean cup         47   0.01
    9             liên_đoàn bóng_đá việt nam vff         43   0.01
   10                     thời hlv park hang seo         39   0.01
   11               lượt hai bảng vòng_loại cuối         35   0.00
   12              hai bảng vòng_loại cuối asian         35   0.00
   13                  night wolf league 2023 24         35   0.00
   14                      thầy trò hlv kim sa

### Bước 7: Tổng hợp tất cả N-grams (Optional)

Nếu muốn lưu tất cả n-grams vào 1 file duy nhất:

In [87]:
# Phân tích và lưu tất cả n-grams vào 1 file
df_all_ngrams = analyze_ngrams_combined(collection, processor, n_docs=1830, max_n=5, top_k=100)

✓ Đã lưu kết quả n-grams (1-5) vào: d:\data\Search_Engine\outputs\ngrams_combined_1830docs_20251009_160118.csv


## 9. Main Execution

### Chạy cell bên dưới để thực hiện toàn bộ quy trình xử lý

In [88]:
def main():
    print("BẮT ĐẦU XỬ LÝ TEXT TỪ MONGODB")
    print("=" * 50)
    
    client, db, collection = connect_to_database()
    if collection is None:
        print("Không thể kết nối database!")
        return
    
    # Xử lý một số documents mẫu
    results = process_all_documents(collection, limit=5)  
    if results:
        analyze_results(results)
        save_results(results, f"processed_vnexpress_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
    
    # Phân tích từ vựng
    processor = VietnameseTextProcessor()
    analyze_multiple_documents(collection, processor, n=50, top_n=50)
    
    # Phân tích n-grams
    analyze_and_save_ngrams(collection, processor, n_docs=1830, max_n=5, top_k=100)

    client.close()
    print("\n✓ Đã đóng kết nối database")
    print("✓ HOÀN THÀNH!")

print("✓ main function đã được định nghĩa")

✓ main function đã được định nghĩa


### Chạy toàn bộ quy trình

Chạy cell bên dưới để thực thi:

In [89]:
def main():
    print("BẮT ĐẦU XỬ LÝ TEXT TỪ MONGODB")
    print("=" * 50)
    
    # Tạo folder outputs
    output_dir = os.path.join("d:\\data\\Search_Engine", "outputs")
    os.makedirs(output_dir, exist_ok=True)
    print(f"✓ Folder outputs: {output_dir}\n")
    
    client, db, collection = connect_to_database()
    if collection is None:
        print("Không thể kết nối database!")
        return
    
    # Xử lý một số documents mẫu
    results = process_all_documents(collection, limit=5)  
    if results:
        analyze_results(results)
        save_results(results, f"processed_vnexpress_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
    
    # Phân tích từ vựng
    processor = VietnameseTextProcessor()
    analyze_multiple_documents(collection, processor, n=50, top_n=50)
    
    # Phân tích từng loại n-gram riêng biệt
    print("\n" + "="*80)
    print("BẮT ĐẦU PHÂN TÍCH N-GRAMS")
    print("="*80)
    
    for n in range(1, 6):
        analyze_single_ngram(collection, processor, n=n, n_docs=1830, top_k=100)
        print("\n")

    client.close()
    print("\n✓ Đã đóng kết nối database")
    print("✓ HOÀN THÀNH!")
    print(f"\n📁 Tất cả kết quả đã được lưu vào folder: {output_dir}")

print("✓ main function đã được định nghĩa")

# Chạy toàn bộ quy trình xử lý
if __name__ == "__main__":
    main()

✓ main function đã được định nghĩa
BẮT ĐẦU XỬ LÝ TEXT TỪ MONGODB
✓ Folder outputs: d:\data\Search_Engine\outputs

✓ Kết nối database thành công!
  Tổng số documents: 1838
Bắt đầu xử lý 5 documents...
✓ Kết nối database thành công!
  Tổng số documents: 1838
Bắt đầu xử lý 5 documents...


Processing documents: 100%|██████████| 5/5 [00:00<00:00, 29.10it/s]

✓ Hoàn thành! Xử lý thành công: 5, Lỗi: 0

PHÂN TÍCH KẾT QUẢ:
Tổng số documents đã xử lý: 5

Thống kê Tokens:
  - Trung bình tokens/document: 727.4
  - Trung bình filtered tokens/document: 541.4
  - Trung bình entities/document: 19.2

Top 10 Đội bóng được nhắc đến nhiều nhất:
  - nam định: 2 lần
  - tuyển việt nam: 2 lần
  - hà nội fc: 1 lần
  - đà nẵng: 1 lần
  - đội tuyển việt nam: 1 lần

Top 10 Cầu thủ được nhắc đến nhiều nhất:
  - Kim Sang: 3 lần
  - Park Hang: 2 lần
  - trẻ cần được thi đấu thường xu: 1 lần
  - nhập tịch vì sử dụng giấy tờ g: 1 lần
  - vào sân: 1 lần
  - bị phạt: 1 lần
  - của Quy định này có đoạn: 1 lần
  - phải được đăng ký đúng quy địn: 1 lần
  - được coi là đủ tư cách thi đấu: 1 lần
  - hợp lệ: 1 lần

Top 10 Giải đấu được nhắc đến nhiều nhất:
  - u23: 4 lần
  - world cup: 3 lần
  - asian cup: 3 lần
  - v-league: 2 lần
  - champions league: 1 lần
  - aff cup: 1 lần


✓ Đã lưu kết quả vào d:\data\Search_Engine\outputs\processed_vnexpress_20251009_160119.json
    Word  Freq  r  Pr(%)  r*Pr
     đội   255  1   1.16 0.012
 cầu_thủ   240  2   1.09 0.022
     nam   239  3   1.08 0.033
  league   233  4   1.06 0.042
    trận   212  5   0.96 0.048
     năm   181  6   0.82 0.049
    việt   180  7   0.82 0.057
     hai   174  8   0.79 0.063
    bóng   170  9   0.77 0.069
     hlv   158 10   0.72 0.072
    giải   148 11   0.67 0.074
     đến   145 12   0.66 0.079
   trước   142 13   0.64 0.084
     anh   139 14   0.63 0.088
     ông   136 15   0.62 0.093
  hà_nội   122 16   0.55 0.089
     mùa   120 17   0.54 0.093
     clb   119 18   0.54 0.097
  nguyễn   118 19   0.54 0.102
     bàn   117 20   0.53 0.106
    phải   117 21   0.53 0.112
    nhất   113 22   0.51 0.113
     văn   112 23   0.51 0.117
      họ   105 24   0.48 0.114
    ngày   102 25   0.46 0.116
    chơi   101 26   0.46 0.119
     hải   101 27   0.46 0.124
   nhiều   100 28   0.45 0.127
    hagl 

Processing 1-gram: 100%|██████████| 1830/1830 [00:26<00:00, 70.16it/s] 



...


TOP 100 3-GRAM
 Rank                     Phrase  Frequency  Pr(%)
    1               kim sang sik        621   0.08
    2           bóng_đá việt nam        612   0.08
    3               hlv kim sang        517   0.07
    4             asean cup 2024        475   0.06
    5              park hang seo        356   0.05
    6           cầu_thủ việt nam        347   0.05
    7               sân hàng đẫy        343   0.05
    8           league 2024 2025        294   0.04
    9               ảnh đức đồng        292   0.04
   10                 clb tp hcm        281   0.04
   11             ảnh hiếu lương        277   0.04
   12              hlv park hang        262   0.04
   13              bùi tiến dũng        259   0.03
   14                đội chủ nhà        248   0.03
   15         đội_tuyển việt nam        242   0.03
   16             tuyển việt nam        241   0.03
   17           nguyễn quang hải        236   0.03
   18               sân việt trì        226   0.03
   19          

Processing 4-gram: 100%|██████████| 1830/1830 [00:24<00:00, 73.67it/s] 




TOP 100 4-GRAM
 Rank                                Phrase  Frequency  Pr(%)
    1                      hlv kim sang sik        517   0.07
    2                     hlv park hang seo        261   0.04
    3              vòng_loại cuối asian cup        140   0.02
    4                    bùi hoàng việt anh        126   0.02
    5                   cuối asian cup 2027        120   0.02
    6                      đội bóng phố núi        112   0.02
    7                      hlv vũ hồng việt        105   0.01
    8                   hlv chu đình nghiêm        103   0.01
    9          vòng_loại hai world_cup 2026        102   0.01
   10                     hlv vũ tiến thành         98   0.01
   11        bóng_đá chuyên_nghiệp việt nam         92   0.01
   12                     mang dòng máu lai         89   0.01
   13                  league 2024 2025 sân         79   0.01
   14                      việt trì phú thọ         78   0.01
   15            liên_đoàn bóng_đá việt nam         77

Processing 5-gram: 100%|██████████| 1830/1830 [00:24<00:00, 73.41it/s]



TOP 100 5-GRAM
 Rank                                     Phrase  Frequency  Pr(%)
    1              vòng_loại cuối asian cup 2027        121   0.02
    2              bảng vòng_loại cuối asian cup         72   0.01
    3 công_ty cổ_phần bóng_đá chuyên_nghiệp việt         71   0.01
    4     cổ_phần bóng_đá chuyên_nghiệp việt nam         71   0.01
    5         bóng_đá chuyên_nghiệp việt nam vpf         68   0.01
    6                       sân việt trì phú thọ         67   0.01
    7                  cầu_thủ mang dòng máu lai         64   0.01
    8                      aff cup nay asean cup         47   0.01
    9             liên_đoàn bóng_đá việt nam vff         43   0.01
   10                     thời hlv park hang seo         39   0.01
   11                  night wolf league 2023 24         35   0.00
   12                      thầy trò hlv kim sang         35   0.00
   13                       trò hlv kim sang sik         35   0.00
   14               lượt hai bảng vòng_loại cu